# Stratégie d'Imputation des Données Manquantes

### 1. Contexte et Approche Stratifiée par Pays

La base de données traitée présente une forte hétérogénéité structurelle due à la diversité des pays inclus.

Afin d'éviter **le biais d'écrasement** qu'aurait provoqué une imputation globale, nous avons opté pour une approche **stratifiée par pays**.

* **Justification :** Un revenu de 2000€ n'a pas la même signification économique ni le même pouvoir prédictif en France qu'en Pologne. De même, les modèles de consommation d'alcool sont fortement culturels.
* **Décision :** Chaque modèle d'imputation a été entraîné et appliqué exclusivement sur les sous-ensembles de données nationaux, garantissant le respect des distributions et des corrélations locales.


### 2. Imputation du Salaire mensuel ('sd_20month_EUR_2020): Approche par Modélisation Prédictive

Pour la variable **Salaire** (variable quantitative continue), nous avons rejeté les imputations univariées simplistes (moyenne/médiane) qui réduisent la variance et faussent les analyses de corrélation ultérieures.

Nous avons privilégié une approche par **Régression Supervisée (Random Forest )** pour les raisons suivantes :

* **Préservation de la Structure de Corrélation :** Le salaire est fortement déterminé par des variables explicatives présentes dans la base. L'utilisation d'un modèle prédictif permet de reconstruire le salaire théorique d'un individu en fonction de son profil socio-démographique précis.
* **Gestion de la Non-Linéarité :** Contrairement à une régression linéaire simple, les modèles d'ensemble comme le randaom Forest capturent les effets de seuil et les interactions complexes.

### 3. Imputation de la Fréquence de Consommation et du Nombre d'enfant de moins de 18 ans dans le ménage : Approche par KNN

Pour les variables **Fréquence de grosse consommation (RSOD_2b)** et **Nombre de mineurs dans le ménage (SD_7)**, nous avons opté pour l'algorithme des **K-Plus Proches Voisins (KNN)**.

Ce choix se justifie par la nature de ces variables et la logique sociologique sous-jacente :

* **Nature des Variables (Ordinales/Discrètes) :**
* La fréquence de consommation est une variable ordinale (échelle de 1 à 10).
* Le nombre de mineurs est une variable discrète.
* L'utilisation d'une régression aurait produit des valeurs continues aberrantes. Le KNN, en faisant la moyenne locale, reste plus cohérent avec des valeurs discrètes.


* **Logique de "Profils Similaires" :** les comportements (boire de l'alcool) et les structures familiales (avoir des enfants) tendent à être partagés par des individus ayant des caractéristiques proches (même âge, même milieu social).


In [ ]:
data = convertir_salaires_en_euro(data)

In [ ]:
data = imputer_salaire_pays(data)

In [ ]:
data = imputer_knn(data)

# Méthodologique : Sélection des Variables Significatives

Afin d'identifier les déterminants spécifiques de la consommation d'alcool pour chaque pays, nous avons adopté une méthodologie de **régression linéaire multiple** avec une procédure de sélection de variables par **élimination descendante (Backward Elimination)**.

Pour chaque pays, la procédure s'initialise par l'ajustement d'un modèle saturé incluant l'ensemble des variables explicatives potentielles. À chaque itération, la significativité statistique de chaque co-variable est évaluée via le **test de Student**. La variable présentant la p-value la plus élevée supérieure au seuil critique fixé à **$\alpha = 0.05$** (et modifiable par argument d'une fonction) est retirée du modèle.Ce dernier est ensuite réajusté sur les variables restantes. 

Ce processus itératif est répété jusqu'à convergence, c'est-à-dire jusqu'à l'obtention d'un modèle final où tous les coefficients conservés sont **statistiquement significatifs**. Cette approche permet d'isoler les facteurs de risque propres à **chaque contexte national**.

Les variables explicatives choisit sont les variables socio-démographiques et de santé de notre base de données. Nous avons éliminés toutes les variables liées à la consommation d'alcool pour éviter tout problème d'endogénéité.

In [ ]:
variables_explicatives= [
    'SD_1',  # Genre
    'SD_2',  # Age
    'SD_4',  # Statut marital
    'SD_6',  # Taille ménage
    'SD_7',  # Mineurs
    'SD_8',  # Urbanisation
    'SD_9',  # Education
    'SD_10', # Activité pro
    "sd_20month_EUR_2020", # Revenu (peut devenir significatif maintenant !)
    "social_class", # Classe sociale
    "CH_1",  # Histoire familiale alcool
    "WB_1",  # Santé physique
    "WB_2",  # Santé mentale
    "WB_3"   # Satisfaction relationnelle
]
resultat = regression_iterative(
df=data, 
target_col='bsqf_alc', 
all_feature_cols= variables_explicatives, 
country_col='COUNTRY', 
liste_pays=['France', 'Poland', 'Bulgaria', 'Iceland'],
seuil_pvalue=0.05)

# Rapport d'Analyse des Déterminants de la Consommation d'Alcool (Approche Prudente)

### 1. Préambule Méthodologique

Les modèles de régression linéaire ci-dessous présentent des coefficients de détermination () faibles, variant de **0,4 % (Bulgarie)** à **6,7 % (Islande)**.

* **Signification :** Les variables socio-démographiques et de bien-être expliquent une part très marginale de la variance de la consommation totale. La consommation d'alcool est donc un comportement très hétérogène, influencé par des facteurs non observés ici (habitudes culturelles, prix, génétique, contexte social immédiat).
* **Portée :** L'échantillon n'étant pas statistiquement représentatif des populations nationales, les résultats suivants décrivent des **corrélations internes à la base de données** et ne doivent pas être généralisés sans réserve à l'ensemble de ces pays.

### 2. Analyse Détaillée 

#### Bilan Global de la Modélisation 

L'analyse des résultats issus de la sélection stepwise mets en évidence deux constats pour l'interprétation : 

* **Un pouvoir explicatif marginal ($R^2$)** : Les coefficients de détermination sont compris entre 0,004 (Bulgarie) et 0,067 (Islande). nous observons que les variables socio-démographiques et de bien-être expliquent une part très faible de la variance (moins de 7 % dans le meilleur des cas, et moins de 1 % pour la Bulgarie). Cela signifie que la consommation volumique d'alcool est un comportement complexe, qui échappe en grande partie aux déterminants sociologiques classiques (âge, sexe, situation familiale).
* **L'absence du revenu** : La variable financière sd_20month_EUR_2020 (Revenu du ménage) a été éliminée par l'algorithme pour l'ensemble des pays, ne franchissant pas le seuil de significativité. Cela veut dire que le niveau de richesse pure n'est pas un prédicteur direct du volume d'alcool consommé.

#### France 

Au sein de l'échantillon français, la consommation semble structurée par des déterminants "classiques" et une variable historique forte.

* **Genre (`SD_1`) :** Avec un coefficient négatif significatif (-301,71), cela signifie que les hommes consommes significativement plus d'alcool que les femmes dans ce groupe.
* **Hérédité sociale (`CH_1`) :** C'est une spécificité marquante des répondants français. La variable "Avoir vécu enfant avec un buveur excessif" (1=Oui, 2=Non) a un coefficient négatif (-360,84). Cela indique que les répondants n'ayant **pas** été exposés à l'alcoolisme familial durant l'enfance consomment significativement moins aujourd'hui.
* **Classe sociale (`social_class`) :** Le coefficient négatif (-34,84) suggère un effet protecteur du statut social : à mesure que l'on s'élève dans la catégorie (de ouvrier vers cadre), le volume déclaré tend à diminuer légèrement.

#### Pologne 
Les répondants polonais présentent des écarts de consommation très brutaux en fonction de leur environnement.

* **Genre (`SD_1`) :** L'effet du genre est massif (-2327,88), bien plus fort qu'en France. Dans cet échantillon, la consommation déclarée est très majoritairement masculine.
* **Urbanisation (`SD_8`) :** Le coefficient positif (+1389,14) indique une corrélation entre la taille de la ville et la consommation. Les répondants des grandes agglomérations déclarent boire davantage que ceux des zones rurales.


* **Structure familiale (`SD_7`) :** La présence de mineurs au foyer est associée positivement à la consommation (+1389,14). Ce résultat contre-intuitif (on s'attendrait à ce que les parents boivent moins) pourrait refléter dans cet échantillon spécifique une consommation domestique plus ancrée ou un profil de répondants plus âgés/installés qui consomment davantage que les jeunes adultes sans enfants.

#### Bulgarie 
Le modèle est extrêmement faible, ce qui indique que pour les répondants bulgares, les variables classiques (âge, sexe) ne prédisent rien. Seule la structure du foyer joue un rôle.

* **Isolement (`SD_6`) :** Le coefficient très négatif (-4660,17) sur la variable "Taille du ménage" (1=Seul, 2=Plusieurs) est crucial. Il indique que le passage de la vie en solo à la vie en groupe fait chuter la consommation. **Dans cet échantillon, les personnes vivant seules sont celles qui déclarent les plus gros volumes.**
* **Statut marital (`SD_4`) :** Le coefficient négatif (-1296,28) montre que les mariés (code 1) consomment plus que les autres statuts (codes élevés). Cela dessine une dichotomie chez les répondants bulgares : une consommation forte chez les isolés, mais aussi une consommation importante chez les couples mariés traditionnels.
* **Éducation (`SD_9`) :** L'éducation agit comme un frein (-2397,77) : les répondants les plus diplômés déclarent des volumes plus faibles.

#### 🇮🇸 Islande : Le lien Santé-Consommation ()

L'échantillon islandais est le mieux expliqué par le modèle, grâce à l'introduction des variables de santé.

* **Le Mal-être (`WB_1`, `WB_2`) :** Les coefficients sont positifs. Puisque les échelles de santé vont de 1 (Très bon) à 5 (Très mauvais), un coefficient positif signifie que **plus la santé physique et mentale déclarée est mauvaise, plus la consommation d'alcool est élevée**. L'alcool apparaît ici corrélé au mal-être.


* **Statut Marital (`SD_4`) :** Contrairement à la Bulgarie, le coefficient positif (+45,35) indique ici que ce sont les personnes seules, divorcées ou veuves (codes élevés) qui consomment plus que les personnes mariées (code 1).
* 
**Éducation (`SD_9`) :** Comme en Bulgarie, un niveau d'éducation élevé est associé à une consommation moindre (-42,52).


### 3. Synthèse Comparative

L'analyse de ces données non-représentatives permet néanmoins de dégager **quatre profils de corrélation** distincts au sein de l'étude :

1. **Le profil socio-culturel (France) :** Où la consommation est liée au genre et à l'histoire familiale.
2. **Le profil environnemental (Pologne) :** Où le lieu de vie (ville) et la composition du foyer (enfants) sont déterminants.
3. **Le profil "Solitaire" (Bulgarie) :** Où vivre seul est le facteur de risque statistique le plus fort identifié.
4. **Le profil "Symptomatique" (Islande) :** Où la consommation est significativement corrélée à une mauvaise auto-évaluation de la santé physique et mentale.

*Note de prudence finale : Ces résultats soulignent des associations statistiques au sein des répondants enquêtés. Ils ne permettent pas d'établir de causalité directe ni de décrire les habitudes de la population générale de ces pays.*